In [ ]:
pip install skorch


Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from skorch import NeuralNetBinaryClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE


In [4]:
# Load data
df = pd.read_csv("framinghamdataset.csv")

X = df.drop(columns=['TenYearCHD'])
y = df['TenYearCHD']

# SMOTE balancing
smote = SMOTE(sampling_strategy=1.0, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_resampled)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_resampled, test_size=0.2, random_state=42)


In [5]:
class FLModel(nn.Module):
    def __init__(self, input_size=15, hidden_units=128, dropout=0.3):
        super(FLModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_units),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_units, hidden_units//2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_units//2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)


In [6]:
net = NeuralNetBinaryClassifier(
    FLModel,
    max_epochs=30,
    optimizer=torch.optim.Adam,
    criterion=nn.BCELoss,
    batch_size=64,
    lr=0.001,
    device='cpu',  # Change to 'cuda' if GPU available
    verbose=0
)


In [7]:
params = {
    'module__hidden_units': [128, 256, 512],
    'module__dropout': [0.2, 0.3, 0.4],
    'lr': [0.001, 0.0005, 0.0001],
    'batch_size': [32, 64],
    'max_epochs': [20, 30, 50],
}


In [ ]:
gs = GridSearchCV(net, params, refit=True, cv=3, scoring='accuracy', verbose=2)
gs.fit(X_train.astype(np.float32), y_train.astype(np.float32))

print(f"✅ Best Hyperparameters: {gs.best_params_}")
print(f"✅ Best CV Accuracy: {gs.best_score_:.4f}")


Fitting 3 folds for each of 162 candidates, totalling 486 fits
[CV] END batch_size=32, lr=0.001, max_epochs=20, module__dropout=0.2, module__hidden_units=128; total time=   6.8s
[CV] END batch_size=32, lr=0.001, max_epochs=20, module__dropout=0.2, module__hidden_units=128; total time=   4.5s
[CV] END batch_size=32, lr=0.001, max_epochs=20, module__dropout=0.2, module__hidden_units=128; total time=   4.5s
[CV] END batch_size=32, lr=0.001, max_epochs=20, module__dropout=0.2, module__hidden_units=256; total time=   5.2s
[CV] END batch_size=32, lr=0.001, max_epochs=20, module__dropout=0.2, module__hidden_units=256; total time=   5.3s
[CV] END batch_size=32, lr=0.001, max_epochs=20, module__dropout=0.2, module__hidden_units=256; total time=   5.0s
[CV] END batch_size=32, lr=0.001, max_epochs=20, module__dropout=0.2, module__hidden_units=512; total time=   5.7s
[CV] END batch_size=32, lr=0.001, max_epochs=20, module__dropout=0.2, module__hidden_units=512; total time=   5.6s
[CV] END batch_si

In [ ]:
y_pred = gs.predict(X_test.astype(np.float32))
accuracy = accuracy_score(y_test, y_pred)
print(f"\n🔥 Test Accuracy with Best Parameters: {accuracy:.4f}")
